In [58]:
import random
import os
from cryptography.hazmat.primitives.hashes import Hash,SHA256
from cryptography.hazmat.backends import default_backend


# https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf

class DSA:
    
    HASH_SIZE = 256
    
    def __init__(self,size_p,size_q):
        # Tamanho dos primos p e q
        pq_pairs = [(512,160),(1024,160),(2048,224),(2048,256),(3072,256)]
        if not (size_p,size_q) in pq_pairs:
            raise ValueError("Not in the list of admitted sizes for p and q:" + str(pq_pairs))
        self.size_p = size_p
        self.size_q = size_q
        self.private = None
        self.public = None
        self.hash = Hash(SHA256(),backend=default_backend())
        
    def generate_keys(self):
        self.Field_q,self.Field_p,self.p,self.q = self.choose_pq_prime()

        self.g = self.generator()
        self.private = random.randint(1,self.q-1) 
        self.public = self.Field_p(pow(self.g,self.private))
    
    def choose_pq_prime(self):
        L = self.size_p
        N = self.size_q
        outlen = DSA.HASH_SIZE
        seedlen = self.size_q
        
        n = ceil(L/outlen) - 1
        b = L - 1 - (n*outlen)
        i = 0
        while True:
            while True:
                domain_parameter_seed = os.urandom(floor(outlen/8))
                self.hash.update(domain_parameter_seed)
                U_aux = int.from_bytes(self.hash.copy().finalize(),'little')
                U = U_aux % (pow(2,(N-1)))
                q = pow(2,N-1) + U + 1 -(U % 2)
                if is_prime(q):
                    break
            offset = 1

            for counter in range(0,4*L):
                Vs = []
                W = 0
                for j in range(0,n+1):
                    x = int((int.from_bytes(domain_parameter_seed,'little') + offset + j))
                    self.hash.update(x.to_bytes(x.bit_length() + 7 // 8,'little'))
                    V_aux = int.from_bytes(self.hash.copy().finalize(),'little')
                    V = V_aux % (pow(2,seedlen))
                    Vs.append(V)
                for i in range(0,len(Vs)-1):
                    W += pow(Vs[i],i*outlen)
                W += (Vs[len(Vs)-1] % (pow(2,b))) * pow(2,n*outlen)
                X = W + pow(2,L-1)
                c = X % (q)
                p = X - (c-1)
                
                if p >= pow(2,L-1) and p <= pow(2,L) and is_prime(p):
                    return FiniteField(q),FiniteField(p),p,q
                offset += n + 1
           

        
    
    def hashing(self,byte):
        
        self.hash.update(byte)
        digest = self.hash.copy().finalize()
        low_bytes = floor(self.size_q / 8)
        return int.from_bytes(digest[:low_bytes], "little")   
    
    def generator(self):
        h =  random.randint(1,self.p-1)
        while True:
            # Ao seguir a recomendação do NIST visto que o gerador não deve ser facilmente relacionado com outro
            e = (self.p-1)/self.q
            generator = pow(int(h),int(e),self.p)
            if generator != 1:
                break
        return generator
    
    def sign(self,message):
        
        self.hash = Hash(SHA256(),backend=default_backend())
        while(True):
            k = random.randint(1,self.q-1)
            r = self.Field_q(self.Field_p(pow(self.g,k) ))
            if r==0:
                continue
            inverse_k = inverse_mod(k,self.q) #calculo da inversa modulo q
            s = self.Field_q((inverse_k * (self.hashing(message) + self.private * r)))
            if s!=0:
                break
        return (r,s)
    
    def verify(self,message,signature,public):
        (r,s) = signature
   
        self.hash = Hash(SHA256(),backend=default_backend())
        if int(r) > int(0) and int(r) < int(self.q) and int(s) > int(0) and int(s) < int(self.q):
            inverse_s = inverse_mod(int(s),self.q)
            u1 = self.Field_q((self.hashing(message) * inverse_s))
            u2 = self.Field_q((int(r) * inverse_s))
            v = self.Field_q(self.Field_p(((pow(self.g,u1)) * (pow(public,u2)))))
            if v != r:
                raise ValueError("Signature is not valid")
        else:
            raise ValueError("Signature values does not fit in parameters size")

dsa = DSA(512,160)
dsa.generate_keys()
(r,s) = dsa.sign(b"eu tenho paciencia para isto mas eu quero dormir")
dsa.verify(b"eu tenho paciencia para isto mas eu quero dormir",(r,s))


In [57]:
import os
import random
from cryptography.hazmat.primitives.hashes import Hash,SHA256
from cryptography.hazmat.backends import default_backend

# https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-5-draft.pdf

class ECDSA:
    
    PRIME_MODULUS = 0
    ORDER = 1
    SEED = 2
    C_SHA = 3
    B_ELEMENT = 4
    X_BASE_POINT = 5
    Y_BASE_POINT = 6
    SECURITY_STRENGTH = 7
    
    def __init__(self,curve):
        # y^2 = x^3 - 3x + b (mod p)
        # (p,n,SEED,c,b,Gx,Gy,security_strength)
        # P <- prime modulus
        # n <- the order
        # SEED <- SHA1 Seed
        # c <- output of SHA1
        # a=-3
        # b <- coefficient satisfying (b^2)*c === -27 mod p
        # Gx <- coordinate x Gx
        # Gy <- coordinate y Gy
        # h = 1
        # secuirity_strength <- segurança quando comparado com cifras simetricas segundo https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-57pt1r4.pdf pag 66 table 2
        prime_curves = { "P-192":(277101735386680763835789423207666416083908700390324961279,6277101735386680763835789423176059013767194773182842284081,0x3045ae6fc8422f64ed579528d38120eae12196d5,0x3099d2bbbfcb2538542dcd5fb078b6ef5f3d6fe2c745de65,0x64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1,0x188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012,0x07192b95ffc8da78631011ed6b24cdd573f977a11e794811,80)
           ,"P-224":(26959946667150639794667015087019630673557916260026308143510066298881,26959946667150639794667015087019625940457807714424391721682722368061,0xbd71344799d5c7fcdc45b59fa3b9ab8f6a948bc5,0x5b056c7e11dd68f40469ee7f3c7a7d74f7d121116506d031218291fb,0xb4050a850c04b3abf54132565044b0b7d7bfd8ba270b39432355ffb4,0xb70e0cbd6bb4bf7f321390b94a03c1d356c21122343280d6115c1d21,0xbd376388b5f723fb4c22dfe6cd4375a05a07476444d5819985007e34,112) 
           ,"P-256":(115792089210356248762697446949407573530086143415290314195533631308867097853951,115792089210356248762697446949407573529996955224135760342422259061068512044369,0xc49d360886e704936a6678e1139d26b7819f7e90,0x7efba1662985be9403cb055c75d4f7e0ce8d84a9c5114abcaf3177680104fa0d,0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b,0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296,0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5,128) 
           ,"P-384":(39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319,39402006196394479212279040100143613805079739270465446667946905279627659399113263569398956308152294913554433653942643,0xa335926aa319a27a1d00896a6773a4827acdac73,0x79d1e655f868f02fff48dcdee14151ddb80643c1406d0ca10dfe6fc52009540a495e8042ea5f744f6e184667cc722483,0xb3312fa7e23ee7e4988e056be3f82d19181d9c6efe8141120314088f5013875ac656398d8a2ed19d2a85c8edd3ec2aef,0xaa87ca22be8b05378eb1c71ef320ad746e1d3b628ba79b9859f741e082542a385502f25dbf55296c3a545e3872760ab7,0x3617de4a96262c6f5d9e98bf9292dc29f8f41dbd289a147ce9da3113b5f0b8c00a60b1ce1d7e819d7a431d7c90ea0e5f,192)
           ,"P-521":(6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151,6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449,0xd09e8800291cb85396cc6717393284aaa0da64ba,0x0b48bfa5f420a34949539d2bdfc264eeeeb077688e44fbf0ad8f6d0edb37bd6b533281000518e19f1b9ffbe0fe9ed8a3c2200b8f875e523868c70c1e5bf55bad637,0x051953eb9618e1c9a1f929a21a0b68540eea2da725b99b315f3b8b489918ef109e156193951ec7e937b1652c0bd3bb1bf073573df883d2c34f1ef451fd46b503f00,0xc6858e06b70404e9cd9e3ecb662395b4429c648139053fb521f828af606b4d3dbaa14b5e77efe75928fe1dc127a2ffa8de3348b3c1856a429bf97e7e31c2e5bd66,0x11839296a789a3bc0045c8a5fb42c7d1bd998f54449579b446817afbd17273e662c97ee72995ef42640c550b9013fad0761353c7086a272c24088be94769fd16650,256)
}
        self.curve_parameters = prime_curves[curve]
        prime = int(self.curve_parameters[ECDSA.PRIME_MODULUS])
        self.curve = EllipticCurve(FiniteField(prime),[0,0,0,-3,self.curve_parameters[ECDSA.B_ELEMENT]])
        self.base_point = ecdsa.curve((ecdsa.curve_parameters[ECDSA.X_BASE_POINT],ecdsa.curve_parameters[ECDSA.Y_BASE_POINT]))
    
    def hashing(self,byte):
        
        self.hash.update(byte)
        digest = self.hash.copy().finalize()
        low_bytes = floor(self.curve_parameters[ECDSA.ORDER] / 8)
        return int.from_bytes(digest[:low_bytes], "little")   
    
    def generate_keys(self):
        N = int(self.curve_parameters[ECDSA.ORDER]).bit_length()
        c = getrandbits(N+64)
        d = (c % (self.curve_parameters[ECDSA.ORDER]-1)) + 1
        Q = d * self.base_point
        # d is the private key and Q is the public key
        self.private_key = d
        self.public_key = Q
        
    def sign(self,message):
        self.hash = Hash(SHA256(),backend=default_backend())
        z = self.hashing(message)
        n = self.curve_parameters[ECDSA.ORDER]
        while True:
            k = random.randint(1,n-1)
            (x1,y1,Z) = k * self.base_point
            r = int(x1) % n
            if r==0:
                continue
            inverse_k = inverse_mod(k,n)
            s = (inverse_k * (z + r*self.private_key)) % n
            if s!=0:
                break
        return (r,s)
    
    def check_public(self,public):
        pass
    
    
    def verify(self,message,signature,public):
        (r,s) = signature
        self.check_public(public)
        n = self.curve_parameters[ECDSA.ORDER]
        if r >= 0 and r <= (n-1) and s >= 1 and s <= (n-1):
            self.hash = Hash(SHA256(),backend=default_backend())
            z = self.hashing(message)
            inverse_s = inverse_mod(s,n)
            u1 = (z*inverse_s) % n
            u2 = (r*inverse_s) % n
            # shamir trick?
            (x1,y1,z) = u1*self.base_point + u2*public
            print(z)
            if z != 0 and ((int(r - x1)) % n) == 0:
                return
            raise ValueError("Signature Invalid")
            

ecdsa = ECDSA("P-224")
ecdsa.generate_keys()
signature = ecdsa.sign(b"oaskdlalskdjasd")
ecdsa.verify(b"oaskdlalskdjasd",signature,ecdsa.public_key)


1


1